# Tuning our CNN 

In [2]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 

In [3]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [4]:
# Load in data
import pandas as pd 
data = pd.read_pickle('data/data.pickle')
X_tfidf = pd.read_pickle('data/tfidf.pickle')


In [5]:
# one hot encoding 
Y = tf.keras.utils.to_categorical(data['class'])
num_classes = len(Y[0])
EPOCHS=5

## Adding F-Score, precision, recall

In [6]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def cnn(filters=3, kernel_size=2, optimizer='sgd', pool_size=2, num_classes=3):
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
    model.add(tf.keras.layers.MaxPool1D(pool_size))
    
    # final fully connected layer 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])
    return model 

model = cnn()
X = np.array(data['tweet_embedded'].to_list())




Using TensorFlow backend.


## Tensorboard

In [11]:
import shutil 
import os

logdir='logs_CNN'
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.fit(X, Y, epochs=EPOCHS, callbacks=[tensorboard_callback])

Train on 24783 samples
Epoch 1/5
24783/24783 [==============================] - 3s 138us/sample - loss: 0.4015 - accuracy: 0.8569 - f1_m: 0.8550 - precision_m: 0.8744 - recall_m: 0.8371
Epoch 2/5
24783/24783 [==============================] - 4s 168us/sample - loss: 0.3940 - accuracy: 0.8599 - f1_m: 0.8574 - precision_m: 0.8755 - recall_m: 0.8405
Epoch 3/5
24783/24783 [==============================] - 6s 256us/sample - loss: 0.3876 - accuracy: 0.8628 - f1_m: 0.8608 - precision_m: 0.8778 - recall_m: 0.8449
Epoch 4/5
24783/24783 [==============================] - 13s 535us/sample - loss: 0.3828 - accuracy: 0.8645 - f1_m: 0.8625 - precision_m: 0.8789 - recall_m: 0.84720s - loss: 0.3902 - accuracy: 0.8631 - f1_m: 0.8600 - preci
Epoch 5/5
24783/24783 [==============================] - 4s 166us/sample - loss: 0.3791 - accuracy: 0.8664 - f1_m: 0.8643 - precision_m: 0.8799 - recall_m: 0.8497


In [15]:
model.fit(X, Y, epochs=EPOCHS, callbacks=[tensorboard_callback], validation_split=.3)

Train on 17348 samples, validate on 7435 samples
Epoch 1/5
17348/17348 [==============================] - 2s 139us/sample - loss: 0.3920 - accuracy: 0.8607 - f1_m: 0.8594 - precision_m: 0.8761 - recall_m: 0.8438 - val_loss: 0.3606 - val_accuracy: 0.8746 - val_f1_m: 0.8714 - val_precision_m: 0.8883 - val_recall_m: 0.8557
Epoch 2/5
17348/17348 [==============================] - 2s 130us/sample - loss: 0.3877 - accuracy: 0.8614 - f1_m: 0.8618 - precision_m: 0.8790 - recall_m: 0.8457 - val_loss: 0.3454 - val_accuracy: 0.8796 - val_f1_m: 0.8769 - val_precision_m: 0.8937 - val_recall_m: 0.8612
Epoch 3/5
17348/17348 [==============================] - 2s 139us/sample - loss: 0.3832 - accuracy: 0.8649 - f1_m: 0.8633 - precision_m: 0.8799 - recall_m: 0.8480 - val_loss: 0.3389 - val_accuracy: 0.8802 - val_f1_m: 0.8801 - val_precision_m: 0.8948 - val_recall_m: 0.8665
Epoch 4/5
17348/17348 [==============================] - 2s 143us/sample - loss: 0.3799 - accuracy: 0.8659 - f1_m: 0.8634 - precisio

In [17]:
def cnn(layers=2, filters=32, kernel_size=2, optimizer='sgd', pool_size=2, num_classes=3):
    model = tf.keras.models.Sequential()
    
    for i in range(layers):
        model.add(tf.keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu'))
        model.add(tf.keras.layers.MaxPool1D(pool_size))
    
    # final fully connected layer 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', f1_m, precision_m, recall_m])
    return model

model = cnn()
X = np.array(data['tweet_embedded'].to_list())
EPOCHS = 100
model.fit(X, Y, epochs=EPOCHS, callbacks=[tensorboard_callback], validation_split=.3)

Train on 17348 samples, validate on 7435 samples
Epoch 1/100
17348/17348 [==============================] - 5s 314us/sample - loss: 0.6747 - accuracy: 0.7573 - f1_m: 0.7447 - precision_m: 0.7590 - recall_m: 0.7404 - val_loss: 0.5524 - val_accuracy: 0.7898 - val_f1_m: 0.7893 - val_precision_m: 0.7921 - val_recall_m: 0.7865
Epoch 2/100
17348/17348 [==============================] - 3s 174us/sample - loss: 0.5550 - accuracy: 0.7853 - f1_m: 0.7833 - precision_m: 0.8010 - recall_m: 0.7670 - val_loss: 0.4521 - val_accuracy: 0.8377 - val_f1_m: 0.8295 - val_precision_m: 0.8580 - val_recall_m: 0.8041
Epoch 3/100
17348/17348 [==============================] - 3s 152us/sample - loss: 0.4864 - accuracy: 0.8197 - f1_m: 0.8153 - precision_m: 0.8401 - recall_m: 0.7927 - val_loss: 0.4192 - val_accuracy: 0.8460 - val_f1_m: 0.8442 - val_precision_m: 0.8666 - val_recall_m: 0.8237
Epoch 4/100
17348/17348 [==============================] - 3s 160us/sample - loss: 0.4556 - accuracy: 0.8299 - f1_m: 0.8280 - 

17348/17348 [==============================] - 2s 126us/sample - loss: 0.1338 - accuracy: 0.9618 - f1_m: 0.9614 - precision_m: 0.9663 - recall_m: 0.9567 - val_loss: 0.6215 - val_accuracy: 0.8067 - val_f1_m: 0.8058 - val_precision_m: 0.8125 - val_recall_m: 0.7995
Epoch 31/100
17348/17348 [==============================] - 2s 123us/sample - loss: 0.1263 - accuracy: 0.9641 - f1_m: 0.9638 - precision_m: 0.9681 - recall_m: 0.9597 - val_loss: 0.5083 - val_accuracy: 0.8421 - val_f1_m: 0.8425 - val_precision_m: 0.8494 - val_recall_m: 0.8359
Epoch 32/100
17348/17348 [==============================] - 2s 123us/sample - loss: 0.1184 - accuracy: 0.9675 - f1_m: 0.9673 - precision_m: 0.9710 - recall_m: 0.9637 - val_loss: 0.5428 - val_accuracy: 0.8459 - val_f1_m: 0.8464 - val_precision_m: 0.8494 - val_recall_m: 0.8434
Epoch 33/100
17348/17348 [==============================] - 2s 131us/sample - loss: 0.1106 - accuracy: 0.9709 - f1_m: 0.9708 - precision_m: 0.9741 - recall_m: 0.9677 - val_loss: 0.5640 

Epoch 60/100
17348/17348 [==============================] - 2s 130us/sample - loss: 0.0159 - accuracy: 0.9987 - f1_m: 0.9988 - precision_m: 0.9988 - recall_m: 0.9987 - val_loss: 0.8797 - val_accuracy: 0.8252 - val_f1_m: 0.8248 - val_precision_m: 0.8268 - val_recall_m: 0.8229
Epoch 61/100
17348/17348 [==============================] - 2s 130us/sample - loss: 0.0148 - accuracy: 0.9989 - f1_m: 0.9989 - precision_m: 0.9989 - recall_m: 0.9989 - val_loss: 0.9601 - val_accuracy: 0.8389 - val_f1_m: 0.8383 - val_precision_m: 0.8397 - val_recall_m: 0.8370
Epoch 62/100
17348/17348 [==============================] - 2s 133us/sample - loss: 0.0139 - accuracy: 0.9992 - f1_m: 0.9992 - precision_m: 0.9993 - recall_m: 0.9991 - val_loss: 0.8986 - val_accuracy: 0.8242 - val_f1_m: 0.8236 - val_precision_m: 0.8254 - val_recall_m: 0.8219
Epoch 63/100
17348/17348 [==============================] - 2s 143us/sample - loss: 0.0135 - accuracy: 0.9991 - f1_m: 0.9991 - precision_m: 0.9991 - recall_m: 0.9991 - val_

Epoch 90/100
17348/17348 [==============================] - 3s 149us/sample - loss: 0.0060 - accuracy: 0.9994 - f1_m: 0.9994 - precision_m: 0.9994 - recall_m: 0.9994 - val_loss: 1.0704 - val_accuracy: 0.8350 - val_f1_m: 0.8340 - val_precision_m: 0.8352 - val_recall_m: 0.8328
Epoch 91/100
17348/17348 [==============================] - 2s 127us/sample - loss: 0.0057 - accuracy: 0.9994 - f1_m: 0.9994 - precision_m: 0.9994 - recall_m: 0.9994 - val_loss: 1.1040 - val_accuracy: 0.8022 - val_f1_m: 0.8020 - val_precision_m: 0.8044 - val_recall_m: 0.7997
Epoch 92/100
17348/17348 [==============================] - 2s 124us/sample - loss: 0.0064 - accuracy: 0.9991 - f1_m: 0.9991 - precision_m: 0.9991 - recall_m: 0.9991 - val_loss: 1.0813 - val_accuracy: 0.8221 - val_f1_m: 0.8212 - val_precision_m: 0.8225 - val_recall_m: 0.8199
Epoch 93/100
17348/17348 [==============================] - 2s 123us/sample - loss: 0.0055 - accuracy: 0.9995 - f1_m: 0.9995 - precision_m: 0.9995 - recall_m: 0.9995 - val_

In [20]:
%tensorboard --logdir='logs_CNN'